## 选择Notebook环境
Notebook的运行环境可以选择conda_pytorch_p36，本实验所用的sagemaker版本为2.42.0，接下来我们会安装此版本的包。

In [ ]:
! pip install --upgrade pip
! pip install sagemaker==2.42.0

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role

region = boto3.session.Session().region_name
role   = get_execution_role()
sess   = sagemaker.Session()
bucket = sess.default_bucket()

bucket

## 准备Docker image

In [ ]:
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name
ecr_repository = 'sagemaker-wenet'

# 登录ECR服务
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com

### 创建容器注册表

In [ ]:
!aws ecr create-repository --repository-name $ecr_repository

### 构建训练镜像

In [ ]:
training_docker_file_path = '/fsx/wenet'

!cat $training_docker_file_path/Dockerfile

In [ ]:
# 构建训练镜像并推送到ECR
tag = ':training-pip-pt181-py38'
training_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)
print('training_repository_uri: ', training_repository_uri)

!cd $training_docker_file_path && docker build -t "$ecr_repository$tag" .
!docker tag {ecr_repository + tag} $training_repository_uri
!docker push $training_repository_uri

# !docker pull $training_repository_uri

### 构建推理镜像

In [ ]:
decoding_docker_file_path='/fsx/wenet/runtime/server/x86'

!cat $decoding_docker_file_path/Dockerfile

In [ ]:
# 构建推理容器并推送到ECR
tag = ':decoding'
decoding_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)
print('decoding_repository_uri: ', decoding_repository_uri)


!cd $decoding_docker_file_path && docker build -t "$ecr_repository$tag" .
!docker tag {ecr_repository + tag} $decoding_repository_uri
!docker push $decoding_repository_uri


## 数据准备

### 数据下载

In [ ]:
data_download_to=/fsx/asr-data/OpenSLR/33
! mkdir -p $data_download_to
! cd /fsx/wenet/examples/aishell/s0 && \
! bash sm-run.sh --stage -1 --stop_stage -1 --data $data_download_to

### 数据预处理 - 本地模式

本地测试模式，可以指定instance_type为local_gpu，如果是CPU实例，则为local。并在fit函数中，指定相关数据的file路径。

In [ ]:
%cd /fsx/wenet

data_dir   = '/opt/ml/input/data/train/asr-data/OpenSLR/33'
trail_dir  = '/opt/ml/input/data/train/sm-train/trail0'
shared_dir = '/opt/ml/input/data/train/sm-train/shared'

from sagemaker.pytorch.estimator import PyTorch

hp= {
    'stage': 0, 'stop_stage': 3, 'train_set':'train', 
    'data': data_dir, 'trail_dir': trail_dir, 'shared_dir': shared_dir
}

estimator=PyTorch(
    entry_point='examples/aishell/s0/sm-run.sh',
    image_uri=training_repository_uri,
    instance_type='local_gpu',
    instance_count=1,
    source_dir='.',
    role=role,
    hyperparameters=hp,
    
    subnets=subnets,
    security_group_ids=security_group_ids,
    
    debugger_hook_config=False,
    disable_profiler=True
)


estimator.fit({'train':'file:///fsx'})


### 数据预处理 - SageMaker托管实例

在托管实例中处理数据，需要将数据来源定义为FSx文件系统。

In [ ]:
from sagemaker.inputs import FileSystemInput
from sagemaker.pytorch.estimator import PyTorch

# 指定文件系统的id.
file_system_id = 'fs-0f8a3b8eef47b6ff8'
# 提供数据集所在的路径，注意格式
file_system_path = '/yobzhbmv'
# 指定挂载文件系统的访问模式，支持"ro"（只读）或"rw"（读写）两种，注意内置算法只支持 以 ro 的方式挂载
file_system_access_mode = 'rw'
# 指定文件系统的类型, 支持"EFS" 或 "FSxLustre"两种.
file_system_type = 'FSxLustre'
# 以VPC内的方式启动 Amazon SageMaker 训练任务,指定所在子网和安全组，subnet需要为list或者tuple格式
security_group_ids = ['sg-04acfc98f6929ee4e']
# subnets= ['vpc-3c49de46']
subnets= ['subnet-07ce0ab63b4cfeb25']

# 定义数据输入
file_system_input_train = FileSystemInput(file_system_id=file_system_id,
                                  file_system_type=file_system_type,
                                  directory_path=file_system_path,
                                  file_system_access_mode=file_system_access_mode)


接下来，我们将启动托管实例执行数据预处理逻辑。

我们需要将instance_type定义为预处理需要的实例类型，fit函数中指定上面定义的FSx文件系统。

In [ ]:
# bash sm-run.sh --stage 0 --stop_stage 3 --train_set train  \
#     --data /opt/ml/input/data/train/asr-data/OpenSLR/33 \
#     --trail_dir /opt/ml/input/data/train/sm-train/trail0 \
#     --shared_dir /opt/ml/input/data/train/sm-train/shared 

# 容器内路径和Notebook实例上的关系如下：
# /opt/ml/input/data/train  <==> /fsx
# /opt/ml/input/data/train/asr-data/OpenSLR/33  <==> /fsx/asr-data/OpenSLR/33
# /opt/ml/input/data/train/sm-train ==> /fsx/sm-train


data_dir   = '/opt/ml/input/data/train/asr-data/OpenSLR/33'
trail_dir  = '/opt/ml/input/data/train/sm-train/trail0'
shared_dir = '/opt/ml/input/data/train/sm-train/shared'


hp= {
    'stage': 0, 'stop_stage': 3, 'train_set':'train', 
    'data': data_dir, 'trail_dir': trail_dir, 'shared_dir': shared_dir
}

estimator=PyTorch(
    entry_point='examples/aishell/s0/sm-run.sh',
    image_uri=training_repository_uri,
    instance_type='ml.c5.4xlarge',
    instance_count=1,
    source_dir='.',
    role=role,
    hyperparameters=hp,
    
    subnets=subnets,
    security_group_ids=security_group_ids,
    
    debugger_hook_config=False,
    disable_profiler=True
)

estimator.fit(inputs={'train': file_system_input_train})

## 模型训练 - 本地训练模式

在模型研发过程中，算法人员需要反复调整代码逻辑，如果每次代码调整就打包一个docker镜像就显得很麻烦，因此，您可以先通过SageMaker的本地训练模式，来调试代码。本地训练模式会直接在Notebook所在实例中启动对应的容器并执行训练逻辑，并自动将数据映射给容器。

CUDA_VISIBLE_DEVICES需要和之行数据处理代码实例的GPU相匹配，如单个实例只有两个GPU卡，则设为'1,2'。

In [ ]:
# bash sm-run.sh --stage 4 --stop_stage 4 --train_set train  \
#     --data /opt/ml/input/data/train/asr-data/OpenSLR/33 \
#     --trail_dir /opt/ml/input/data/train/sm-train/trail0 \
#     --shared_dir /opt/ml/input/data/train/sm-train/shared 

instance_type='local_gpu'
instance_count = 1
CUDA_VISIBLE_DEVICES='0'

hp= {
    'stage': 4, 'stop_stage': 4, 'train_set':'train', 
    'data': data_dir, 'trail_dir': trail_dir, 'shared_dir': shared_dir,
    'CUDA_VISIBLE_DEVICES': CUDA_VISIBLE_DEVICES, 
    'num_nodes': instance_count
}

estimator=PyTorch( 
    entry_point='examples/aishell/s0/sm-run.sh',
    image_uri=training_repository_uri,
    instance_type =instance_type,
    instance_count=instance_count,
    source_dir='.',
    role=role,
    hyperparameters=hp,
    
    subnets=subnets,
    security_group_ids=security_group_ids,
    
    debugger_hook_config=False,
    disable_profiler=True
)


estimator.fit({'train': 'file:///fsx'})

## 模型训练 - SageMaker托管实例

在确定代码逻辑无误后，我们可以很容易通过修改参数的方式，使用托管的实例开启真正的训练任务。
这里，我们只需要调整实例类型、需要的实例数量及数据输入方式。我们以2台ml.p3.8xlarge的实例为例，其各自包含4张Nvidia V100显卡，共8张显卡。

In [ ]:
# bash sm-run.sh --stage 4 --stop_stage 4 --train_set train  \
#     --data /opt/ml/input/data/train/asr-data/OpenSLR/33 \
#     --trail_dir /opt/ml/input/data/train/sm-train/trail0 \
#     --shared_dir /opt/ml/input/data/train/sm-train/shared 

instance_count = 2

instance_type='ml.p3.8xlarge'
CUDA_VISIBLE_DEVICES='0,1,2,3'

# instance_type='ml.p3.2xlarge'
# CUDA_VISIBLE_DEVICES='0'

hp= {
    'stage': 4, 'stop_stage': 4, 'train_set':'train', 
    'data': data_dir, 'trail_dir': trail_dir, 'shared_dir': shared_dir,
    'CUDA_VISIBLE_DEVICES': CUDA_VISIBLE_DEVICES, 
    'ddp_init_protocol': 'tcp',
    'num_nodes': instance_count
}

estimator=PyTorch( 
    entry_point='examples/aishell/s0/sm-run.sh',
    image_uri=training_repository_uri,
    instance_type =instance_type,
    instance_count=instance_count,
    source_dir='.',
    role=role,
    hyperparameters=hp,
    
    subnets=subnets,
    security_group_ids=security_group_ids,
    
    debugger_hook_config=False,
    disable_profiler=True,
    environment={
        'NCCL_SOCKET_IFNAME': 'eth0',
        'NCCL_IB_DISABLE': 1
    }
)


estimator.fit(inputs={'train': file_system_input_train})


### 导出模型

In [ ]:
%cd /fsx/wenet

In [ ]:
from sagemaker.pytorch.estimator import PyTorch

instance_type='local'
instance_count = 1
CUDA_VISIBLE_DEVICES='0'

hp= {
    'stage': 6, 'stop_stage': 6, 'train_set':'train', 
    'data': data_dir, 'trail_dir': trail_dir, 'shared_dir': shared_dir,
    'decode_modes': 'attention_rescoring',
    'average_num': 30,
    'CUDA_VISIBLE_DEVICES': CUDA_VISIBLE_DEVICES, 
#     'ddp_init_protocol': 'tcp',
#     'num_nodes': instance_count
}

estimator=PyTorch(
    entry_point='examples/aishell/s0/sm-run.sh',
    image_uri=training_repository_uri,
    instance_type =instance_type,
    instance_count=instance_count,
    source_dir='.',
    role=role,
    hyperparameters=hp,
    
    subnets=subnets,
    security_group_ids=security_group_ids,
    
    debugger_hook_config=False,
    disable_profiler=True,
    environment={
        'NCCL_SOCKET_IFNAME': 'eth0',
        'NCCL_IB_DISABLE': 1
    }
)

estimator.fit({'train':'file:///fsx'})